# 04. Agent Workflow with LangChain/LangGraph

This notebook covers:
- Building agents with LangChain
- Creating workflows with LangGraph
- Tool integration for agents
- Stateful agent conversations

## 1. Import Libraries

In [ ]:
import sys
sys.path.append('/workspace')

from src.utils.db_utils import DatabaseConnection, get_database_context
from src.utils.text2sql_utils import execute_text2sql
from src.utils.embedding_utils import search_similar_documents

from langchain_community.llms import Ollama
from langchain.agents import Tool, AgentExecutor, create_react_agent
from langchain.prompts import PromptTemplate
from langchain.memory import ConversationBufferMemory
import pandas as pd

print("✓ Libraries imported successfully")

## 2. Initialize Components

In [ ]:
# Initialize database
db = DatabaseConnection()

# Initialize LLM
import os
ollama_host = os.getenv('OLLAMA_HOST', 'http://localhost:11434')
llm = Ollama(base_url=ollama_host, model="llama2")

print("✓ Components initialized")

## 3. Create Agent Tools

In [ ]:
# Tool 1: Execute SQL query from natural language
def text2sql_tool(query: str) -> str:
    """Convert natural language to SQL and execute"""
    result = execute_text2sql(db, query, log_execution=True)
    if result['success']:
        return f"Query executed successfully. Results:\n{result['results'].to_string()}"
    else:
        return f"Error: {result['error']}"

# Tool 2: Search documentation
def search_docs_tool(query: str) -> str:
    """Search for relevant documentation"""
    results = search_similar_documents(db, query, limit=2)
    docs_text = ""
    for _, title, content, similarity in results:
        docs_text += f"\n{title}:\n{content}\n"
    return docs_text

# Tool 3: Get database schema
def get_schema_tool(input: str = "") -> str:
    """Get database schema information"""
    return get_database_context()

# Create tools list
tools = [
    Tool(
        name="Text2SQL",
        func=text2sql_tool,
        description="Use this to query the database using natural language. Input should be a question about the data."
    ),
    Tool(
        name="SearchDocs",
        func=search_docs_tool,
        description="Use this to search documentation and guides. Input should be a search query."
    ),
    Tool(
        name="GetSchema",
        func=get_schema_tool,
        description="Use this to get database schema information."
    )
]

print(f"✓ Created {len(tools)} tools for the agent")

## 4. Create ReAct Agent

In [ ]:
# Create agent prompt
template = """Answer the following questions as best you can. You have access to the following tools:

{tools}

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!

Question: {input}
Thought: {agent_scratchpad}
"""

prompt = PromptTemplate.from_template(template)

# Create agent
agent = create_react_agent(llm, tools, prompt)
agent_executor = AgentExecutor(
    agent=agent,
    tools=tools,
    verbose=True,
    handle_parsing_errors=True,
    max_iterations=5
)

print("✓ Agent created successfully")

## 5. Test Agent with Simple Query

In [ ]:
# Test the agent
question = "How many employees are in the Engineering department?"

print(f"Question: {question}\n")
try:
    response = agent_executor.invoke({"input": question})
    print(f"\nFinal Answer: {response['output']}")
except Exception as e:
    print(f"Error: {e}")

## 6. Test Agent with Multi-Step Query

In [ ]:
# More complex question requiring multiple steps
question = "What is the total salary cost for the Engineering department, and what percentage of the department budget is it?"

print(f"Question: {question}\n")
try:
    response = agent_executor.invoke({"input": question})
    print(f"\nFinal Answer: {response['output']}")
except Exception as e:
    print(f"Error: {e}")

## 7. Agent with Memory (Stateful Conversation)

In [ ]:
# Create agent with conversation memory
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)

# Note: For proper memory integration, you would need to modify the agent creation
# This is a simplified example
print("Memory-enabled agents require more advanced setup with LangGraph")
print("See next section for LangGraph implementation")

## 8. Enhanced LangGraph Workflow with Decision Loops


In [ ]:
from langgraph.graph import StateGraph, END
from typing import TypedDict, Annotated
import operator

# Define enhanced state with quality tracking
class AgentState(TypedDict):
    messages: Annotated[list, operator.add]
    query: str
    sql: str
    results: str
    iteration_count: int
    quality_score: float
    should_retry: bool
    max_iterations: int

# Define nodes
def analyze_query(state: AgentState) -> AgentState:
    """Analyze the user query"""
    query = state['query']
    state['messages'].append(f"[Iteration {state['iteration_count']}] Analyzing query: {query}")
    return state

def generate_sql(state: AgentState) -> AgentState:
    """Generate SQL from query"""
    result = execute_text2sql(db, state['query'], log_execution=False)
    state['sql'] = result.get('sql_query', '')
    state['results'] = str(result.get('results', 'No results'))
    state['messages'].append(f"[Iteration {state['iteration_count']}] Generated SQL: {state['sql']}")
    
    # Update iteration count
    state['iteration_count'] += 1
    return state

def evaluate_results(state: AgentState) -> AgentState:
    """Evaluate the quality of results"""
    # Simple quality heuristics
    quality_score = 0.0
    
    # Check if SQL was generated
    if state['sql'] and state['sql'].strip():
        quality_score += 0.4
    
    # Check if results were returned
    if 'No results' not in state['results'] and state['results']:
        quality_score += 0.3
        
        # Check result length (more results might indicate better query)
        if len(state['results']) > 50:
            quality_score += 0.2
    
    # Check for common SQL keywords
    sql_lower = state['sql'].lower()
    if any(keyword in sql_lower for keyword in ['select', 'from', 'where', 'join']):
        quality_score += 0.1
    
    state['quality_score'] = quality_score
    state['messages'].append(f"[Iteration {state['iteration_count']-1}] Quality score: {quality_score:.2f}")
    
    return state

def decide_next_step(state: AgentState) -> str:
    """Decide whether to retry or finish"""
    # Retry conditions
    should_retry = (
        state['quality_score'] < 0.7 and  # Quality threshold
        state['iteration_count'] < state['max_iterations'] and  # Max iterations not reached
        state['sql']  # At least some SQL was generated
    )
    
    state['should_retry'] = should_retry
    
    if should_retry:
        state['messages'].append(f"Quality insufficient, retrying... ({state['iteration_count']}/{state['max_iterations']})")
        return "retry"
    else:
        state['messages'].append("Quality sufficient or max iterations reached, finishing...")
        return "finish"

def refine_query(state: AgentState) -> AgentState:
    """Refine the query for retry"""
    # Add more context or hints to the query
    state['messages'].append(f"[Iteration {state['iteration_count']}] Refining query for better results")
    # In a real implementation, you might add more schema hints or examples here
    return state

def format_response(state: AgentState) -> AgentState:
    """Format the final response"""
    state['messages'].append(f"Final results after {state['iteration_count']} iteration(s): {state['results'][:100]}...")
    return state

# Create graph
workflow = StateGraph(AgentState)

# Add nodes
workflow.add_node("analyze", analyze_query)
workflow.add_node("generate", generate_sql)
workflow.add_node("evaluate", evaluate_results)
workflow.add_node("refine", refine_query)
workflow.add_node("format", format_response)

# Add edges
workflow.set_entry_point("analyze")
workflow.add_edge("analyze", "generate")
workflow.add_edge("generate", "evaluate")

# Conditional edge: retry or finish based on quality
workflow.add_conditional_edges(
    "evaluate",
    decide_next_step,
    {
        "retry": "refine",
        "finish": "format"
    }
)

# Loop back to generate after refinement
workflow.add_edge("refine", "generate")
workflow.add_edge("format", END)

# Compile
app = workflow.compile()

print("✓ Enhanced LangGraph workflow with decision loops created")
print("Features:")
print("  - Quality evaluation after each SQL generation")
print("  - Automatic retry with refinement if quality is low")
print("  - Maximum iteration limit to prevent infinite loops")
print("  - Iteration tracking and detailed logging")

In [ ]:
# Test the enhanced workflow with decision loops
initial_state = {
    "messages": [],
    "query": "Show me all employees in Sales department",
    "sql": "",
    "results": "",
    "iteration_count": 1,
    "quality_score": 0.0,
    "should_retry": False,
    "max_iterations": 3  # Maximum 3 attempts
}

print(f"Input Query: {initial_state['query']}")
print(f"Max Iterations: {initial_state['max_iterations']}\n")

final_state = app.invoke(initial_state)

print("\nWorkflow execution log:")
print("=" * 80)
for msg in final_state['messages']:
    print(f"  {msg}")

print("\n" + "=" * 80)
print(f"Final Quality Score: {final_state['quality_score']:.2f}")
print(f"Total Iterations: {final_state['iteration_count']}")
print(f"\nGenerated SQL:\n{final_state['sql']}")
print(f"\nFinal Results:\n{final_state['results'][:200]}...")

## Summary

In this notebook, you learned:
- ✓ How to create agents with LangChain
- ✓ How to build custom tools for agents
- ✓ How to create ReAct agents for reasoning
- ✓ How to build workflows with LangGraph
- ✓ How to manage state in agent workflows

Next: Move to `05_visualization.ipynb` to create charts from query results.